This is another workflow for blue book for bulldozers. I will be trying to use the KNNImputer

In [17]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_log_error

from catboost import CatBoostRegressor

In [5]:
df = pd.read_csv("../data/Blue Book for Bulldozers/TrainAndValid.csv", low_memory=False)

In [9]:
for label in df:
    if df[label].isna().sum() > 1:
        print(label, df[label].isna().mean())

auctioneerID 0.04879112571420264
MachineHoursCurrentMeter 0.6425861041245656
UsageBand 0.8214917445686677
fiSecondaryDesc 0.3409926871465333
fiModelSeries 0.8578452039990502
fiModelDescriptor 0.8187148956379725
ProductSize 0.5248511017741787
Drive_System 0.7405197020581636
Enclosure 0.0008093085016161939
Forks 0.5209208670747132
Pad_Type 0.8034979573441112
Ride_Control 0.6299279376202453
Stick 0.8034979573441112
Transmission 0.5444441213671983
Turbocharged 0.8034979573441112
Blade_Extension 0.9370411293488217
Blade_Width 0.9370411293488217
Enclosure_Type 0.9370411293488217
Engine_Horsepower 0.9370411293488217
Hydraulics 0.20006154621539238
Pushblock 0.9370411293488217
Ripper 0.7408637793253178
Scarifier 0.9370144754760139
Tip_Control 0.9370411293488217
Tire_Size 0.7634153787999942
Coupler 0.46527727296958066
Coupler_System 0.8910244294859679
Grouser_Tracks 0.8912643143412374
Hydraulics_Flow 0.8912643143412374
Track_Type 0.7523782523782524
Undercarriage_Pad_Width 0.7506263660109814
Stic

There are a lot of missing values. KNNImputer might not work at all. But lets try. Why not?

In [10]:
df.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,fiBaseModel,fiSecondaryDesc,fiModelSeries,fiModelDescriptor,ProductSize,fiProductClassDesc,state,ProductGroup,ProductGroupDesc,Drive_System,Enclosure,Forks,Pad_Type,Ride_Control,Stick,Transmission,Turbocharged,Blade_Extension,Blade_Width,Enclosure_Type,Engine_Horsepower,Hydraulics,Pushblock,Ripper,Scarifier,Tip_Control,Tire_Size,Coupler,Coupler_System,Grouser_Tracks,Hydraulics_Flow,Track_Type,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000.0,999089,3157,121,3.0,2004,68.0,Low,11/16/2006 0:00,521D,521,D,NaN,NaN,NaN,Wheel Loader - 110.0 to 120.0 Horsepower,Alabama,WL,Wheel Loader,NaN,EROPS w AC,None or Unspecified,NaN,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 Valve,NaN,NaN,NaN,NaN,None or Unspecified,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000.0,117657,77,121,3.0,1996,4640.0,Low,3/26/2004 0:00,950FII,950,F,II,NaN,Medium,Wheel Loader - 150.0 to 175.0 Horsepower,North Carolina,WL,Wheel Loader,NaN,EROPS w AC,None or Unspecified,NaN,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 Valve,NaN,NaN,NaN,NaN,23.5,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000.0,434808,7009,121,3.0,2001,2838.0,High,2/26/2004 0:00,226,226,NaN,NaN,NaN,NaN,Skid Steer Loader - 1351.0 to 1601.0 Lb Operat...,New York,SSL,Skid Steer Loaders,NaN,OROPS,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Auxiliary,NaN,NaN,NaN,NaN,NaN,None or Unspecified,None or Unspecified,None or Unspecified,Standard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500.0,1026470,332,121,3.0,2001,3486.0,High,5/19/2011 0:00,PC120-6E,PC120,NaN,-6E,NaN,Small,"Hydraulic Excavator, Track - 12.0 to 14.0 Metr...",Texas,TEX,Track Excavators,NaN,EROPS w AC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 Valve,NaN,NaN,NaN,NaN,NaN,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000.0,1057373,17311,121,3.0,2007,722.0,Medium,7/23/2009 0:00,S175,S175,NaN,NaN,NaN,NaN,Skid Steer Loader - 1601.0 to 1751.0 Lb Operat...,New York,SSL,Skid Steer Loaders,NaN,EROPS,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Auxiliary,NaN,NaN,NaN,NaN,NaN,None or Unspecified,None or Unspecified,None or Unspecified,Standard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df.drop("SalesID", axis=1, inplace=True)

In [12]:
cat_labels = [c for c in df if not pd.api.types.is_numeric_dtype(df[c])]
num_labels = [c for c in df if pd.api.types.is_numeric_dtype(df[c]) and c != "SalePrice"]

In [18]:
cat_imputer = Pipeline([
    ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),
    ("ohe", OneHotEncoder(handle_unknown="ignore"))
])

In [19]:
num_imputer = Pipeline([
    ("imputer", KNNImputer())
])

In [20]:
preprocessor = ColumnTransformer([
    ("cat_imputer", cat_imputer, cat_labels),
    ("num_imputer", num_imputer, num_labels)
], remainder="passthrough")

In [21]:
model = Pipeline([
    ("preprocessor", preprocessor),
    ("model", CatBoostRegressor(verbose=False))
])

In [22]:
X, y = df.drop("SalePrice", axis=1), df["SalePrice"]

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((330158, 51), (330158,), (82540, 51), (82540,))

In [29]:
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat_imputer',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['UsageBand', 'saledate',
                                                   'fiModelDesc', 'fiBaseModel',
                                                   'fiSecondaryDesc',
                                                   'fiModelSeries',
                                                   'fiModelDescriptor',
                                                   'Product...
                                                   'Blade_Width',
                                                   'Enclosure_Type',
                                                   'Engine_Horsepower',
                                                   'Hydraulics', 'Pushblock',
                                                   'Ripper', 'Scarifier',
                                                   'Tip_Control', 'Tire_Size', ...]),
                                                 ('num_imputer',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer())]),
                                                  ['MachineID', 'ModelID',
                                                   'datasource', 'auctioneerID',
                                                   'YearMade',
                                                   'MachineHoursCurrentMeter'])])),
                ('model',
                 <catboost.core.CatBoostRegressor object at 0x000001F313B24BB0>)])

This took way tooooo long

In [30]:
model.score(X_test, y_test)

0.816736968624715

Let me try GridSearchCv 

In [32]:
grid_params = {
    "preprocessor__cat_imputer__imputer__strategy": ["constant", "most_frequent"],
    "preprocessor__num_imputer__imputer__n_neighbors": [3, 6, 8, 10],
    "preprocessor__num_imputer__imputer__add_indicator": [True, False],
    "preprocessor__num_imputer_imputer__weights": ["distance", "uniform"]
}

In [35]:
gs_model = GridSearchCV(model, grid_params, cv=5, n_jobs=-1, scoring="neg_root_mean_squared_error")

In [36]:
gs_model.fit(X_train, y_train)

ValueError: Invalid parameter 'num_imputer_imputer' for estimator ColumnTransformer(remainder='passthrough',
                  transformers=[('cat_imputer',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('ohe',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['UsageBand', 'saledate', 'fiModelDesc',
                                  'fiBaseModel', 'fiSecondaryDesc',
                                  'fiModelSeries', 'fiModelDescriptor',
                                  'ProductSize', 'fiProductClassDesc', 'stat...
                                  'Pad_Type', 'Ride_Control', 'Stick',
                                  'Transmission', 'Turbocharged',
                            